In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dask
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
# Import libraries
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

train_dataframes = []
for i in tqdm(range(1, 2)):
    train_dataframes.append(pd.read_csv(f'data-train/train_data_part_{i}.csv'))
train_data = pd.concat(train_dataframes, ignore_index=True)

# free up memory by deleting the dataframes we no longer need
del train_dataframes

# This code reads the data from a CSV file named "products_data.csv" into a pandas DataFrame
products_data = pd.read_csv('data-train/products_data.csv')

# This code reads the data from a CSV file named "test_data.csv" into a pandas DataFrame
test_data = pd.read_csv('data-train/test_data.csv')

100%|██████████| 1/1 [00:32<00:00, 32.70s/it]
C:\Users\jeann\AppData\Local\Temp\ipykernel_3184\3163275204.py:15: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  products_data = pd.read_csv('data-train/products_data.csv')


In [3]:

columns_to_keep = ['date', 'transaction_id', 'customer_id', 'product_id']
train_data = train_data[columns_to_keep]

In [4]:
# Create a new DataFrame with customer_id, product_id, and count
frequency_df = train_data.groupby(['customer_id', 'product_id'])['transaction_id'].count().reset_index(name='frequency')
# Créer la variable cible (produit suivant)
train_data = train_data.sort_values(by=['customer_id', 'date'])
train_data['next_product'] = train_data.groupby('customer_id')['product_id'].shift(-1)


In [5]:
# Retirer les lignes où la cible est manquante
train_data = train_data.dropna(subset=['next_product'])

# Fusionner la fréquence des achats
train_features = train_data.merge(frequency_df, on=['customer_id', 'product_id'], how='left')

# Préparer les caractéristiques (X) et la cible (y)
X_train = train_features[['customer_id', 'product_id', 'frequency']]
y_train = train_features['next_product']



In [6]:
#X_train.sample(5)
y_train.sample(5)

4034261    Product_23916
5855985    Product_60501
1291280    Product_56156
852251     Product_25388
8162538    Product_11440
Name: next_product, dtype: object

In [7]:
# Identifier les clients présents dans le train
train_customers = set(train_data['customer_id'])

# Filtrer les données de test pour ne garder que les clients présents dans le train
test_data = test_data[test_data['customer_id'].isin(train_customers)]

# Vérifier le résultat
print(f"Nombre de clients dans le test après filtrage : {test_data['customer_id'].nunique()}")


Nombre de clients dans le test après filtrage : 10000


In [8]:
from sklearn.preprocessing import LabelEncoder

# Encodage des colonnes catégoriques
customer_encoder = LabelEncoder()
product_encoder = LabelEncoder()

X_train['customer_id'] = customer_encoder.fit_transform(X_train['customer_id'])
X_train['product_id'] = product_encoder.fit_transform(X_train['product_id'])
#y_train['customer_id'] = customer_encoder.transform(y_train['customer_id'])
y_train = product_encoder.fit_transform(y_train)

# Encodage des test_data
test_data['customer_id'] = customer_encoder.transform(test_data['customer_id'])
test_data['product_id'] = product_encoder.transform(test_data['product_id'])


C:\Users\jeann\AppData\Local\Temp\ipykernel_3184\2238909656.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['customer_id'] = customer_encoder.fit_transform(X_train['customer_id'])
C:\Users\jeann\AppData\Local\Temp\ipykernel_3184\2238909656.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['product_id'] = product_encoder.fit_transform(X_train['product_id'])


entrainer une foret aleatoire

In [10]:
# Séparer les données en train et validation
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [11]:
# Créer et entraîner le modèle
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)


In [13]:
from joblib import dump, load

# Entraîner le modèle
rf_model.fit(X_train_split, y_train_split)

# Sauvegarder le modèle
dump(rf_model, 'rf_model.joblib')

# Charger le modèle si nécessaire
rf_model = load('rf_model.joblib')


MemoryError: could not allocate 3375824896 bytes

In [12]:

rf_model.fit(X_train_split, y_train_split)

MemoryError: could not allocate 6751649792 bytes

In [9]:

# Évaluer le modèle sur les données de validation
y_val_pred = rf_model.predict(X_val)

print("Accuracy sur les données de validation :", accuracy_score(y_val, y_val_pred))
print("Rapport de classification :\n", classification_report(y_val, y_val_pred))


MemoryError: could not allocate 27006599168 bytes

Trop de données, on ne peux pas réaliser ce modèle